In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.19.0
pandas 1.0.5
sklearn 0.23.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
loaded_keras_model = keras.models.load_model(
    './graph_def_and_weights/fashion_mnist_model.h5')
loaded_keras_model(np.ones((1, 28, 28)))

<tf.Tensor: id=491, shape=(1, 10), dtype=float32, numpy=
array([[0.18206659, 0.06282796, 0.1108905 , 0.05255988, 0.07318188,
        0.01522345, 0.10224032, 0.22811368, 0.12305682, 0.04983885]],
      dtype=float32)>

In [4]:
# 1、 keras model 转成 tflite
# tf.lite.TFLiteConverter.from_keras_model()
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
keras_tflite = keras_to_tflite_converter.convert()

In [5]:
if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
with open('./tflite_models/keras_tflite', 'wb') as f:
    f.write(keras_tflite)

In [6]:
# 2、concrete funtion 转成 tflite
# tf.lite.TFLite.from_concrete_function()
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype))

In [7]:
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype=np.float32)))

<tf.Tensor: id=613, shape=(1, 10), dtype=float32, numpy=
array([[0.18206659, 0.06282796, 0.1108905 , 0.05255988, 0.07318188,
        0.01522345, 0.10224032, 0.22811368, 0.12305682, 0.04983885]],
      dtype=float32)>

In [8]:
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_tflite = concrete_func_to_tflite_converter.convert()


In [9]:
with open('./tflite_models/concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

In [10]:
# 3、saved model 转成 tflite
# tf.lite.TFLiteConverter.from_saved_model()
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('./keras_saved_graph/')
saved_model_tflite = saved_model_to_tflite_converter.convert()

In [11]:
with open('./tflite_models/saved_model_tflite', 'wb') as f:
    f.write(saved_model_tflite)

